In [20]:
import numpy as np
import pandas as pd
import spacy
import string
import pickle as pkl

#load in the data
train_df = pd.read_csv('snli_train.tsv', sep="\t")
val_df = pd.read_csv('snli_val.tsv',sep="\t")

#get data & target values
train_sentence1=train_df['sentence1'].values.tolist()
train_sentence2=train_df['sentence2'].values.tolist()
#train_label=train_df['label'].values.tolist()
val_sentence1=val_df['sentence1'].values.tolist()
val_sentence2=val_df['sentence2'].values.tolist()
#val_label=val_df['label'].values.tolist()


In [21]:
# Load English tokenizer, tagger, parser, NER and word vectors
tokenizer = spacy.load('en_core_web_sm')
punctuations = string.punctuation

# lowercase and remove punctuation
def tokenize(sent):
  tokens = tokenizer(sent)
  return [token.text.lower() for token in tokens if (token.text not in punctuations)]

In [ ]:
# This is the code cell that tokenizes train/val/test datasets
def tokenize_dataset(dataset):
    token_dataset = []
    # we are keeping track of all tokens in dataset 
    # in order to create vocabulary later
    all_tokens = []
    
    for sample in dataset:
        tokens = tokenize(sample)
        token_dataset.append(tokens)
        all_tokens += tokens

    return token_dataset, all_tokens

#train set tokens
print ("Tokenizing train data")
train_sentence1_tokens, train_all_sentence1_tokens = tokenize_dataset(train_sentence1)
train_sentence2_tokens, train_all_sentence2_tokens = tokenize_dataset(train_sentence2)
#pkl.dump(train_sentence1_tokens, open("train_sentence1_tokens.p", "wb"))
#pkl.dump(train_sentence2_tokens, open("train_sentence2_tokens.p", "wb"))
pkl.dump(train_all_sentence1_tokens, open("train_all_sentence1_tokens.p", "wb"))
pkl.dump(train_all_sentence2_tokens, open("train_all_sentence2_tokens.p", "wb"))

#val set tokens
print ("Tokenizing val data")
#val_sentence1_tokens, _ = tokenize_dataset(val_sentence1)
#val_sentence2_tokens, _ = tokenize_dataset(val_sentence2)
#pkl.dump(val_sentence1_tokens, open("val_sentence1_tokens.p", "wb"))
#pkl.dump(val_sentence2_tokens, open("val_sentence2_tokens.p", "wb"))

Tokenizing train data


In [23]:
print("test")

test
